<a href="https://colab.research.google.com/github/CHIRCHIR-C/CHIRCHIR-C/blob/main/Copy_of_st_cms_v2_0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- COLAB SETUP: Mount Drive and Install Packages ---
import os
from google.colab import drive

# 1. Mount Google Drive
# This is required to access your data and save results persistently.
# Run this cell, follow the prompt, and select your Google account.
print("Mounting Google Drive...")
drive.mount('/content/drive')

# 2. Install Required Libraries
# We consolidate all 'pip install' commands here using '!pip'.
# PyMuPDF (fitz) and PyPDF2 are necessary for the script.
print("Installing necessary packages...")
!pip install --quiet PyMuPDF PyPDF2 pandas

# Note: The 'fitz' import is available after installing PyMuPDF
# The 'PyPDF2' import is available after installing PyPDF2

Mounting Google Drive...
Mounted at /content/drive
Installing necessary packages...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 86.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 14.9 MB/s eta 0:00:00


In [ ]:
# --- PDF MERGER LOGIC (Adapted for Google Drive) ---
import fitz
from pathlib import Path
from datetime import datetime
from PyPDF2 import PdfMerger
import pandas as pd
import re
import os

# Replace 'Your_Drive_Folder' with the actual path in your Google Drive.
# Example: '/content/drive/MyDrive/Publications/2025'
# Base directory where the subfolders (like '07') are located
BASE_DRIVE_FOLDER = '/content/drive/MyDrive/Vanguard Analytics/General/Assignments/The Star/Publications/2025'
base_path = Path(BASE_DRIVE_FOLDER)

# Define the persistent output directory for the merged PDF
# Example: '/content/drive/MyDrive/Reports/Merged_Dailies'
OUTPUT_DRIVE_FOLDER = '/content/drive/MyDrive/Vanguard Analytics/General/Assignments/The Star/Publications/Merged'
output_dir = Path(OUTPUT_DRIVE_FOLDER)
output_dir.mkdir(parents=True, exist_ok=True) # Ensure output directory exists

# Prompt user for the subfolder name
subfolder_name = input("Enter the name of the subfolder to merge the PDFs (e.g., '07'): ")
subfolder_path = base_path / subfolder_name

# Check if the subfolder exists
if not subfolder_path.exists() or not subfolder_path.is_dir():
    print(f"Error: The source folder '{subfolder_name}' does not exist at {base_path}")
else:
    # Filter for PDF files in the specified subfolder
    pdffiles = [str(f) for f in subfolder_path.glob("*.pdf") if f.is_file()]

    if not pdffiles:
        print(f"No PDF files found in the folder '{subfolder_name}'.")
    else:
        # Get the current period (YYYYMM)
        current_date_period = input("Enter the Period (YYYYMM, e.g., 202507): ")

        # Create the result file name
        result_file = f"st_{current_date_period}_STITCHED.pdf"

        # Create a PdfMerger object
        merger = PdfMerger()

        # Append the PDF files
        print(f"Merging {len(pdffiles)} PDF files...")
        for pdf in pdffiles:
            merger.append(pdf)

        # Write the merged result file
        merged_file = output_dir / result_file
        merger.write(str(merged_file))
        merger.close()

        print(f"\n✅ Merged file '{result_file}' successfully created in {OUTPUT_DRIVE_FOLDER}")

Enter the name of the subfolder to merge the PDFs (e.g., '07'): 09
Enter the Period (YYYYMM, e.g., 202507): 202509
Merging 8 PDF files...

✅ Merged file 'st_202509_STITCHED.pdf' successfully created in /content/drive/MyDrive/Vanguard Analytics/General/Assignments/The Star/Publications/Merged


In [ ]:
# system Tesseract (Colab)
!apt-get update -qq
!apt-get install -y -qq tesseract-ocr libtesseract-dev

# python libs
!pip install -q pymupdf pandas pytesseract pillow rapidfuzz


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.6.0-1ubuntu1.5_amd64.deb ...
Unpacking libarchive-dev:amd64 (3.6.0-1ubuntu1.5) ...
Selecting previously unselected package libleptonica-dev.
Preparing to unpack .../libleptonica-dev_1.82.0-3build1_amd64.deb ...
Unpacking libleptonica-dev (1.82.0-3build1) ...
Selecting previously unselected package libtesseract-dev:amd64.
Preparing to unpack .../libtesseract-dev_4.1.1-2.1build1_amd64.deb ...
Unpacking libtesseract-dev:amd64 (4.1.1-2.1build1) ...
Setting up libleptonica-dev (1.82.0-3build1) ...
Setting up libarchive-dev:amd64 (3.6.0-1ubuntu1.5) ...
Setting up libtesseract-dev:amd64 (4.1.1-2.1build1) ...
Processing triggers for ma

In [ ]:
# The Star Hybrid Extractor v4 (layout + reporter-driven)
# Label: st_mpe_1.0
import fitz
import re
import io
import os
import logging
import pandas as pd
import pytesseract
from PIL import Image
from rapidfuzz import process, fuzz

# ---------------- CONFIG ----------------
PDF_PATH = "/content/drive/MyDrive/Vanguard Analytics/General/Assignments/The Star/Publications/2025/09/EPAPER0109_compressed.pdf"
REPORTERS_CSV_DRIVE_PATH = "/content/drive/MyDrive/Vanguard Analytics/General/Assignments/The Star/Reporters_List/Reporters_v1.0.csv"
OUTPUT_CSV_DRIVE_PATH = "/content/drive/MyDrive/Vanguard Analytics/General/Assignments/The Star/Reports/EPAPER0109_STITCHED_compressed_v4.0.csv"
#DEBUG_SAMPLE_CSV = "/content/drive/MyDrive/Vanguard Analytics/General/Assignments/The Star/Reports/EPAPER0109_STITCHED_debug_sample_v4.0.csv"

OCR_LANG = "eng"
MIN_SELECTABLE_ALPHA = 80
SECTION_FUZZY_THRESHOLD = 75
REPORTER_FUZZY_THRESHOLD = 82

# utility function
# sanitize any list before joining it
def safe_join(tokens):
    clean = []
    for t in tokens:
        if isinstance(t, str):
            clean.append(t.strip())
        elif pd.notna(t):
            clean.append(str(t).strip())
    clean = [c for c in clean if c]   # remove empty strings
    return " ".join(clean)

# your expected sections
expected_news_sections_st = [
    'News', 'POLITICS', 'PHOTO STORY','BUSINESS PICTORIAL', 'CORRIDORS OF POWER',
    'NEWS GENERAL', 'NEWSBUSINESS','BIG READ','★letters','SPORTS FEATURE', '★comment','★letters','society...',
    'TRADE', 'VOICES', 'NEWS COUNTIES','COUNTIES','CLASSIFIED',
    'BUSINESS CONSULTANT', 'NIGHT OF COOL', 'BUSINESS', 'Sports','SPORTS LOCAL','FOOTBALL','ATHLETICS','COVER STORY',
    'EDITORIAL', 'OPINION', 'LIFESTYLE', 'PUZZLES', 'WORLD NEWS', 'LETTERS', 'COLUMNIST','COMMENTARY','COLUMN'#,'theSTAR'
]
EXPECTED_SECTIONS_UP = [s.upper() for s in expected_news_sections_st]

# *opic keywords (expandable later with full list)
TOPIC_KEYWORDS = {
    "Health Systems & Policy": [
        # Core Systems & Bodies
        "universal health coverage", "uhc", "sha", "social health authority",
        "shif", "social health insurance fund", "nhif", "health policy",
        "moh", "ministry of health", "kemsa", "kmtc", "pharmacy and poisons board",
        "nacada", "kmpdu", "doctors union", "clinical officer", "pharmaceutical",

        # Facilities & Care
        "referral hospital", "level 4", "level 5", "dispensary", "clinic",
        "maternity", "icu", "hdu", "dialysis", "chemotherapy", "palliative",
        "medical bill", "insurance claim", "access to care",

        # Public Health Issues
        "malaria", "cholera", "tb", "hiv", "aids", "pandemic", "outbreak",
        "vaccination", "immunization", "mental health", "nutrition",
        "maternal health", "infant mortality", "community health promoter"
    ],

    "Crime": [
        # Law Enforcement Agencies
        "police", "dci", "directorate of criminal investigations", "nps",
        "eacc", "ethics and anti-corruption", "ippoa", "odpp",
        "inspector general", "atpu", "anti-terror", "security operation",
        "traffic police", "rungu", "gsu",

        # Judicial/Legal
        "arrest", "suspect", "court", "magistrate", "judge", "prosecution",
        "bail", "bond", "ruling", "jail", "prison", "remand", "lawsuit",
        "petition", "affidavit", "injunction", "sentence", "convicted",
        "charge sheet", "case file",

        # Offenses
        "murder", "homicide", "femicide", "assault", "robbery", "theft",
        "burglary", "fraud", "graft", "corruption", "bribe", "embezzlement",
        "money laundering", "trafficking", "drug bust", "raid", "gang",
        "gunshot", "shootout", "abduction", "kidnap", "losing money", "carjacking"
    ],

    "Business": [
        # Finance & Markets
        "market", "economy", "finance", "bond", "nse", "nairobi securities exchange",
        "shareholders", "dividend", "stocks", "equity", "tbills", "treasury bills",
        "recession", "economic growth", "trading", "capital markets",

        # Banking & Currency
        "cbk", "central bank", "shilling", "forex", "dollar", "exchange rate",
        "inflation", "interest rate", "loan", "credit", "bank", "lender", "fintech",
        "mobile money", "mpesa", "atm", "digital currency", "banking sector",

        # Corporate & Trade
        "revenue", "profit", "loss", "turnover", "audit", "tax", "kra",
        "kenya revenue authority", "file returns", "customs", "imports",
        "exports", "manufacturing", "epz", "chamber of commerce",
        "supply chain", "consumer price index", "agribusiness",

        # Energy & Infrastructure
        "fuel price", "epra", "kplc", "token", "electricity", "infrastructure",
        "sgr", "cargo", "logistics", "highway", "port", "pipeline", "oil prices",
        "renewable energy", "geothermal"
    ],

    "Sports": [
        # Football
        "football", "match", "soccer", "premier league", "kpl", "fkf",
        "harambee stars", "gor mahia", "afc leopards", "caf", "afcon",
        "fifa", "cecafa", "super league", "uefa", "champions league",
        "transfer window", "score", "penalty", "red card", "goal",

        # Athletics
        "athletics", "marathon", "sprint", "steeplechase", "diamond league",
        "olympics", "world athletics", "kip keino", "relays", "track and field",
        "world record", "doping", "runner", "gold medal", "cross country",

        # Other Sports
        "rugby", "sevens", "shujaa", "lionesses", "volleyball", "malkia strikers",
        "rally", "wrc", "safari rally", "basketball", "cricket", "golf",
        "tournament", "championship", "coach", "squad", "fixture", "venue",
        "boxing", "swimming"
    ],

    "Education": [
        # Institutions & Bodies
        "school", "university", "college", "tvet", "ministry of education",
        "tsc", "teachers service commission", "knut", "kuppet", "helb",
        "kuccps", "kicd", "unions", "board of management",

        # Curriculum & Exams
        "kcse", "kcpe", "kpsea", "cbc", "competency based curriculum",
        "8-4-4", "junior secondary", "jss", "grade 6", "grade 9", "form four",
        "syllabus", "national exam", "marking scheme", "results slip",

        # Academic Life
        "graduation", "intake", "fees", "capitation", "bursary", "scholarship",
        "chancellor", "vice chancellor", "headteacher", "principal", "lecturer",
        "strike", "learning", "literacy", "admissions", "campus", "tuition"
    ],

    "Opinion": [
        # Formats
        "opinion", "column", "editorial", "op-ed", "commentary", "analysis",
        "perspective", "viewpoint", "my view", "letter to the editor",
        "star say", "nation view", "standard editorial", # Common specific column names

        # Tone/Context keywords
        "expert take", "guest writer", "contributor", "thought leadership",
        "analysis", "critique", "in my view", "basing my argument", "ponder",
        "reflect", "rejoinder"
    ]
}
TOPIC_KEYWORDS_LO = {k: [w.lower() for w in v] for k, v in TOPIC_KEYWORDS.items()}

# ---------------- logging ----------------
logging.basicConfig(level=logging.INFO, format="%(asctime)s — %(levelname)s — %(message)s")
logger = logging.getLogger(__name__)

# ---------------- Utilities ----------------
def page_has_selectable_text(page, min_alpha=MIN_SELECTABLE_ALPHA):
    txt = page.get_text("text")
    if not txt:
        return False
    alpha = sum(1 for c in txt if c.isalpha())
    return alpha >= min_alpha

def ocr_page(page, dpi=200):
    mat = fitz.Matrix(dpi/72, dpi/72)
    pix = page.get_pixmap(matrix=mat, alpha=False)
    img = Image.open(io.BytesIO(pix.tobytes()))
    text = pytesseract.image_to_string(img, lang=OCR_LANG)
    return text

def normalize_page_text_for_matching(text):
    # minimal normalization for matching: collapse whitespace, remove weird spacing in uppercase headers
    t = re.sub(r'\s+', ' ', text)
    t = re.sub(r'(\b[A-Z])\s(?=[A-Z])', r'\1', t)  # join spaced uppercase words like B R I E F LY -> BRIEFLY
    return t.strip()

def extract_page_number_star(text, page_index):
    lines = text.split("\n")
    header = " ".join(lines[:5]).strip()  # Only inspect top 5 text lines

    # --- PAGE 1 RULE ---
    if page_index == 0:
        return 1  # The paper number is always 1 even if not printed

    # --- RULE: integer at beginning (page 2,4,6…)
    start_num = re.match(r"^\s*(\d{1,2})\b", header)
    if start_num:
        return int(start_num.group(1))

    # --- RULE: integer at end (page 3,5,7…)
    end_num = re.search(r"\b(\d{1,2})\s*$", header)
    if end_num:
        return int(end_num.group(1))

    return None

def extract_header(text, page_index=None):
    lines = [l.strip() for l in text.splitlines() if l.strip()]
    head = " ".join(lines[:8]) if lines else ""
    # date
    date_m = re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+[A-Za-z]+\s+\d{1,2},\s+\d{4}", head)
    date = date_m.group(0) if date_m else None
    # page number
    # --- NEW: Use The-Star-specific page-number extractor ---
    page_no = extract_page_number_star(text, page_index) if page_index is not None else None
    # publication
    pub_m = re.search(r"THE[-\s]*STAR|the-star\.co\.ke", head, re.IGNORECASE)
    pub = pub_m.group(0) if pub_m else None
    # section fuzzy match
    section = None
    if head:
        best = process.extractOne(head.upper(), EXPECTED_SECTIONS_UP, scorer=fuzz.partial_ratio)
        if best and best[1] >= SECTION_FUZZY_THRESHOLD:
            section = best[0]
    return {"date": date, "page_number": page_no, "publication": pub, "news_section": section}

# ---------------- Block / layout helpers ----------------
# ===== BEGIN REPLACE: split_blocks_layout (stable, smaller sub-blocks) =====
def split_blocks_layout(page):
    """
    Safer block splitter that:
      - extracts text spans per block,
      - splits large blocks on double-newline into sub-blocks,
      - returns list of dicts: {text, bbox, block, x0, y0, x1, y1}
    """
    d = page.get_text("dict")
    blocks = []
    for b in d.get("blocks", []):
        if b.get("type") != 0:
            continue
        # build raw text from spans (preserve line breaks)
        lines = []
        for ln in b.get("lines", []):
            line_text = "".join(span.get("text", "") for span in ln.get("spans", []))
            lines.append(line_text)
        raw = "\n".join(lines).strip()
        if not raw:
            continue
        # split on two or more newlines to get logical sub-blocks
        subs = re.split(r'\n{2,}', raw)
        for sub in subs:
            s = sub.strip()
            if not s:
                continue
            # bbox sometimes is None or not list — try-catch
            bbox = b.get("bbox") if isinstance(b.get("bbox"), (list, tuple)) and len(b.get("bbox")) >= 4 else [0, 0, 0, 0]
            x0, y0, x1, y1 = bbox[0], bbox[1], bbox[2], bbox[3]
            blocks.append({
                "text": s,
                "bbox": bbox,
                "block": b,
                "x0": float(x0 or 0),
                "y0": float(y0 or 0),
                "x1": float(x1 or 0),
                "y1": float(y1 or 0),
            })
    # Sort by page reading order (y then x)
    blocks = sorted(blocks, key=lambda bb: (bb["y0"], bb["x0"]))
    return blocks
# ===== END REPLACE: split_blocks_layout =====

# ===== BEGIN REPLACE: group_blocks_by_column (gap clustering, no sklearn) =====
def group_blocks_by_column(blocks, column_gap_threshold=None):
    """
    Backwards-compatible, robust column grouping for The Star newspaper.

    - Accepts optional column_gap_threshold (keeps older callers happy).
    - If column_gap_threshold is None, uses a sensible default (35 px).
    - Safely extracts x0,y0 from bbox and clusters by horizontal gaps.
    - Limits columns to MAX_COLS (5).
    - Returns {col_index: [blocks_sorted_top_to_bottom]}
    """
    if not blocks:
        return {}

    # Allow caller override; otherwise use default for The Star
    DEFAULT_GAP = 35.0
    COLUMN_GAP = float(column_gap_threshold) if column_gap_threshold is not None else DEFAULT_GAP
    MAX_COLS = 5

    # ---- Safe extractor for bbox coordinates ----
    def safe_xy(b):
        bbox = b.get("bbox")
        if bbox and isinstance(bbox, (list, tuple)) and len(bbox) >= 2:
            try:
                return float(bbox[0]), float(bbox[1])
            except Exception:
                return 0.0, 0.0
        return 0.0, 0.0

    # Annotate blocks with x0,y0
    annotated = []
    for b in blocks:
        x0, y0 = safe_xy(b)
        annotated.append({"x0": x0, "y0": y0, "blk": b})

    # Sort left -> right by x0
    annotated.sort(key=lambda x: x["x0"])

    # If after sorting all x0 are nearly identical (OCR fallback), put everything in one column
    x_vals = [a["x0"] for a in annotated]
    if len(x_vals) > 1 and max(x_vals) - min(x_vals) < 1.0:
        # single column, sort top to bottom
        col_sorted = sorted(annotated, key=lambda x: (x["y0"], x["x0"]))
        return {0: [x["blk"] for x in col_sorted]}

    # Cluster into columns by horizontal gap
    columns = []
    current = [annotated[0]]
    last_x = annotated[0]["x0"]

    for item in annotated[1:]:
        x0 = item["x0"]
        if (x0 - last_x) > COLUMN_GAP and len(columns) < MAX_COLS:
            columns.append(current)
            current = [item]
        else:
            current.append(item)
        last_x = x0
    columns.append(current)

    # If we accidentally created more than MAX_COLS due to weird spacing, merge nearest columns left-to-right
    while len(columns) > MAX_COLS:
        # find smallest gap between consecutive column centers and merge those two
        centers = []
        for col in columns:
            xs = [it["x0"] for it in col]
            centers.append(sum(xs) / len(xs) if xs else 0)
        min_gap = None
        merge_idx = 0
        for i in range(len(centers)-1):
            gap = abs(centers[i+1] - centers[i])
            if min_gap is None or gap < min_gap:
                min_gap = gap
                merge_idx = i
        # merge merge_idx and merge_idx+1
        columns[merge_idx] = columns[merge_idx] + columns[merge_idx+1]
        del columns[merge_idx+1]

    # Now sort each column top-to-bottom and produce the map
    col_map = {}
    for i, col in enumerate(columns):
        col_sorted = sorted(col, key=lambda x: (x["y0"], x["x0"]))
        col_map[i] = [x["blk"] for x in col_sorted]

    return col_map

# ===== END REPLACE: group_blocks_by_column =====

def span_is_bold(span):
    # flag bit 2 indicates bold in PyMuPDF spans (heuristic)
    return (span.get("flags", 0) & 2) != 0

def extract_title_smart(blk):
    """
    Improved title extractor for The Star newspaper.
    Handles:
      - bold spans split across multiple lines
      - multi-line titles
      - titles with short first lines
      - fallback to first 2–3 lines if bold detection fails
    """
    if not blk or not blk.get("block"):
        return None

    block = blk["block"]
    title_lines = []

    # --- STEP 1: Try bold spans anywhere in the block (not only top lines) ---
    bold_fragments = []
    for line in block.get("lines", []):
        for span in line.get("spans", []):
            txt = span.get("text", "").strip()
            if not txt:
                continue
            # Bold or slightly larger font indicates title
            if span_is_bold(span) or span.get("size", 0) >= 11.0:
                bold_fragments.append(txt)

    if bold_fragments:
        title = " ".join(bold_fragments).strip()
        # Require only 2+ words for a valid title
        if len(title.split()) >= 2 and len(title) <= 150:
            return title

    # --- STEP 2: If bold failed, use first 3 lines as candidate title area ---
    raw_lines = [l.strip() for l in blk["text"].splitlines() if l.strip()]

    if not raw_lines:
        return None

    # take first 3 non-empty lines
    candidate = " ".join(raw_lines[:3]).strip()

    # remove byline-like patterns
    candidate = re.sub(r"^\s*BY\s+.+", "", candidate, flags=re.IGNORECASE)

    # ensure at least 2 words and reasonable length
    if len(candidate.split()) >= 2 and len(candidate) <= 150:
        return candidate

    # --- STEP 3: Last resort: first non-empty line with ≥2 words ---
    for ln in raw_lines:
        if len(ln.split()) >= 2 and len(ln) <= 150:
            return ln

    return None

def is_likely_byline(line):
    ln = line.strip()
    if not ln:
        return False
    if ln.upper().startswith("BY ") and len(ln.split()) <= 12:
        return True
    if ln.isupper() and 2 <= len(ln.split()) <= 6 and len(ln) < 80:
        return True
    return False

# --- REVERTED/ORIGINAL detect_photo_credit_anywhere for plain text ---
import re

def detect_photo_credit_anywhere(block):
  """
  Robust universal photo credit detector for The Star:
  - detects uppercase or proper-case names
  - detects '/ENOS TECHE', '/Faith Matete', '/PCS', '/FILE', '/HANDOUT'
  - supports slash appearing anywhere in the block
  - supports broken spans, irregular spacing
  """

  # --- Extract text from spans if block dict provided ---
  if isinstance(block, dict) and "block" in block:
    spans = []
    for line in block["block"].get("lines", []):
      for span in line.get("spans", []):
        txt = span.get("text", "")
        spans.append(txt)
    text = " ".join(spans)
  else:
    text = str(block) if isinstance(block, str) else ""

  # Normalize whitespace
  text = re.sub(r"\s+", " ", text)

  # Combined robust pattern:
  # /NAME
  # / FILE
  # / HANDOUT
  # / Faith Matete
  # /Enos TecHe (case-insensitive)
  photo_regex = re.compile(
    r"/\s*(?:"
    r"[A-Z][A-Za-z]+(?:\s+[A-Z][A-Za-z]+)*"  # Proper case name (Faith Matete)
    r"|[A-Z]{2,}"                # All caps (PCS, FILE)
    r"|FILE|HANDOUT|PCS|ENOS\s+TECHE"      # explicit common variants
    r")",
    flags=re.IGNORECASE
  )

  m = photo_regex.search(text)
  if m:
    # Normalize spacing inside the credit
    return re.sub(r"\s+", " ", m.group(0)).strip()

  return None

# --- NEW HELPER FOR BOLD PHOTO CREDIT FROM BLOCKS ---
def detect_bold_photo_credit_from_block(blk_obj):
    """
    Detects a photo credit in bold spans within a PyMuPDF block object.
    Now supports delimiters '/' and 'COMPILED BY '.
    """
    if not blk_obj or not blk_obj.get("block"):
        return None

    # Regex to detect either "/" or "COMPILED BY "
    # followed by uppercase words, proper-case names, or known variants
    credit_regex = re.compile(
        r"(?:/|COMPILED BY )\s*(?:"
        r"[A-Z][A-Za-z]+(?:\s+[A-Z][A-Za-z]+)*"     # Proper case names
        r"|[A-Z]{2,}"                                # All caps words
        r"|FILE|HANDOUT|PCS|ENOS\s+TECHE"            # Explicit known variants
        r")",
        flags=re.IGNORECASE
    )

    for line in blk_obj["block"].get("lines", []):
        for span in line.get("spans", []):
            if span_is_bold(span):
                text_in_span = span.get("text", "").strip()
                m = credit_regex.search(text_in_span)
                if m:
                    return re.sub(r"\s+", " ", m.group(0)).strip()

    return None

# ---------------- Reporter loading & matching (from DN logic) ----------------
def load_reporters(path):
    # Load CSV or Excel
    if path.lower().endswith(tuple([".xls", ".xlsx"])):
        df = pd.read_excel(path)
    else:
        df = pd.read_csv(path)

    # Validate required columns
    expected_cols = ["Reporter", "Reporter_Category", "Publication_Section"]
    missing = [c for c in expected_cols if c not in df.columns]
    if missing:
        raise ValueError(f"Reporters CSV missing expected columns: {missing}")

    # ---- CLEANING BLOCK (Fix for numpy.float64 / NaN issues) ----
    df["Reporter"] = df["Reporter"].astype(str)           # convert all to string
    df["Reporter"] = df["Reporter"].str.strip()           # trim spaces
    df = df[df["Reporter"].str.lower() != "nan"]          # drop 'nan' strings
    df = df[df["Reporter"] != ""]                         # drop empty strings
    df = df.dropna(subset=["Reporter"])                   # drop actual NaN
    # --------------------------------------------------------------

    # Normalize reporter names
    df["Reporter_norm"] = (
        df["Reporter"]
        .astype(str)
        .str.strip()
        .replace(r"\s+", " ", regex=True)    # unwarp multi spaces
    )

    df["Reporter_norm_lower"] = df["Reporter_norm"].str.lower()

    # Output clean lists
    reporters_list = df["Reporter_norm"].tolist()         # proper names
    reporters_list_lower = df["Reporter_norm_lower"].tolist()

    return df, reporters_list, reporters_list_lower

REPORTERS_DF, REPORTERS_LIST, REPORTERS_LIST_LOWER = load_reporters(REPORTERS_CSV_DRIVE_PATH)
logger.info(f"Loaded {len(REPORTERS_LIST)} reporters from CSV.")

def create_name_pattern_flexible(reporter_name):
    parts = reporter_name.split()
    if not parts:
        return re.compile(r'(?!x)x')
    pattern_str = r'\s*'.join([re.escape(word) for word in parts])
    return re.compile(pattern_str, re.IGNORECASE)

def normalize_byline_text(raw_byline):
    if not raw_byline:
        return []
    s = raw_byline.strip()
    s = re.sub(r'^\s*BY[.\s:,-]*', '', s, flags=re.IGNORECASE)
    s = s.replace("/", " ").replace("—", " ").replace("–", " ").strip()
    s = re.sub(r'\s{2,}', ' ', s)
    parts = re.split(r'\s+(?:AND|&|,|;)\s+|\n', s, flags=re.IGNORECASE)
    parts = [p.strip() for p in parts if p and len(p.strip())>1]
    seen = set(); out = []
    for p in parts:
        key = re.sub(r'\s+', ' ', p).strip()
        if key.lower() not in seen:
            out.append(key); seen.add(key.lower())
    return out

def match_single_name(name_candidate):
    if not name_candidate:
        return None, None, None, 0
    cand = re.sub(r'\s+', ' ', name_candidate.strip())
    cand_lower = cand.lower()
    try:
        idx = REPORTERS_LIST_LOWER.index(cand_lower)
    except ValueError:
        idx = None
    if idx is not None:
        row = REPORTERS_DF.iloc[idx]
        return row["Reporter_norm"], row.get("Reporter_Category", None), row.get("Publication_Section", None), 100
    best = process.extractOne(cand_lower, REPORTERS_LIST_LOWER, scorer=fuzz.WRatio)
    if best and best[1] >= REPORTER_FUZZY_THRESHOLD:
        matched_lower = best[0]
        idx = REPORTERS_LIST_LOWER.index(matched_lower)
        row = REPORTERS_DF.iloc[idx]
        return row["Reporter_norm"], row.get("Reporter_Category", None), row.get("Publication_Section", None), best[1]
    return None, None, None, 0

def match_reporter(raw_byline):
    candidates = normalize_byline_text(raw_byline)
    matched_names=[]; cats=[]; pubs=[]; scores=[]
    for cand in candidates:
        mname, mcat, mpub, score = match_single_name(cand)
        matched_names.append(mname); cats.append(mcat); pubs.append(mpub); scores.append(score)
    return {"matched_names": matched_names, "reporter_categories": cats, "publication_sections": pubs, "match_scores": scores}

# ---------------- Topic matching ----------------
def match_topic(text, section_hint=None):
    if not text: return "other"
    low = text.lower()
    for topic, keywords in TOPIC_KEYWORDS_LO.items():
        for kw in sorted(keywords, key=lambda x: -len(x)):
            if kw in low:
                return topic
    # fallback by section hint
    if section_hint:
        sh = (section_hint or "").lower()
        if "sport" in sh: return "Sports"
        if "business" in sh: return "Business"
    return "other"

# ---------------- Hybrid page -> article logic ----------------
import re
# Assuming pandas (pd) is imported elsewhere for REPORTERS_DF, and logger is available.
# The full function relies on several external functions (e.g., split_blocks_layout, match_reporter, etc.)
# which are assumed to be defined elsewhere in your script.

# --- Helper Function to Get All Known Credit Terms ---
# NOTE: This relies on REPORTERS_LIST being available (e.g., globally or passed in).
import re

# Assume KNOWN_CREDITS helper is defined as before
def get_known_credits_list():
    """Returns a list of all names/sources that should be in the photo_credit column."""
    known_list = list(REPORTERS_LIST) if 'REPORTERS_LIST' in globals() else []
    known_list.extend(["HANDOUT", "FILE", "AP", "REUTERS", "MPE", "STAR", "COURTESY"])
    return known_list

# --- The Core Photo Credit Cleanup Function (Updated) ---
def clean_photo_credit(credit_text):
    """
    Cleans up the raw photo credit string aggressively by retaining only the
    first matched known name/source and immediately cutting off all following text.
    """
    if not credit_text:
        return None

    # Normalize the text (uppercase, strip leading/trailing slashes and spaces)
    normalized_credit = credit_text.strip().upper().strip('/')

    KNOWN_CREDITS = get_known_credits_list()

    # 1. Create a dynamic regex pattern for all known core credit terms
    # The pattern looks for the term followed by an optional space/separator
    # and then captures EVERYTHING ELSE (.*) to be replaced.

    # Create the search group: (FILE|HANDOUT|TEDDY MULEI|...)
    credit_group = '|'.join(re.escape(c.upper()) for c in KNOWN_CREDITS if c)

    # Create the pattern: Look for a word boundary, the credit term, an optional separator/space,
    # and then capture everything that follows.
    # The (?:...) non-capturing group is for the optional slash or space after the credit.
    # Example: (\bFILE\s?)(.*)

    # Look for the core credit at the start or after a space/slash.
    match = re.search(r'\b(' + credit_group + r')\b(?:/|\s|$)', normalized_credit)

    if match:
        matched_term = match.group(1).strip()

        # Aggressive Cutoff: Find the position immediately after the matched term
        # and forcefully return only the text up to that point.

        # Calculate the position right after the matched credit term
        start_pos = match.start(1)
        end_pos = match.end(1)

        # We need to ensure we capture the full word, and nothing more.
        clean_text = normalized_credit[start_pos:end_pos].strip()

        # Perform final cleanup to remove trailing words like 'BY' or a slash, but only if they
        # immediately follow the term in the original raw text.

        # Remove common prefixes from the clean result if they were accidentally included
        for prefix in ['BY', 'FOR', 'FROM']:
             if clean_text.startswith(prefix + ' '):
                 clean_text = clean_text[len(prefix) + 1:].strip()

        return clean_text

    # 2. Fallback: If no known credit term was found
    for prefix in ['BY', 'FOR', 'FROM', 'PHOTO:']:
        if normalized_credit.startswith(prefix):
            normalized_credit = normalized_credit[len(prefix):].strip()

    # Return the first 3 words of the fallback
    return ' '.join(normalized_credit.split()[:3]).strip().strip('/')


# --- The Main Article Extraction Function ---
import re
# Assuming pandas (pd) is imported elsewhere for REPORTERS_DF, and logger is available.
# The full function relies on several external functions (e.g., split_blocks_layout, match_reporter, etc.)
# which are assumed to be defined elsewhere in your script.

# --- Helper Function to Get All Known Credit Terms ---
# NOTE: This relies on REPORTERS_LIST being available (e.g., globally or passed in).
import re

# Assume KNOWN_CREDITS helper is defined as before
def get_known_credits_list():
    """Returns a list of all names/sources that should be in the photo_credit column."""
    known_list = list(REPORTERS_LIST) if 'REPORTERS_LIST' in globals() else []
    known_list.extend(["HANDOUT", "FILE", "AP", "REUTERS", "MPE", "STAR", "COURTESY"])
    return known_list

# --- The Core Photo Credit Cleanup Function (Updated) ---
def clean_photo_credit(credit_text):
    """
    Cleans up the raw photo credit string aggressively by retaining only the
    first matched known name/source and immediately cutting off all following text.
    """
    if not credit_text:
        return None

    # Normalize the text (uppercase, strip leading/trailing slashes and spaces)
    normalized_credit = credit_text.strip().upper().strip('/')

    KNOWN_CREDITS = get_known_credits_list()

    # 1. Create a dynamic regex pattern for all known core credit terms
    # The pattern looks for the term followed by an optional space/separator
    # and then captures EVERYTHING ELSE (.*) to be replaced.

    # Create the search group: (FILE|HANDOUT|TEDDY MULEI|...)
    credit_group = '|'.join(re.escape(c.upper()) for c in KNOWN_CREDITS if c)

    # Create the pattern: Look for a word boundary, the credit term, an optional separator/space,
    # and then capture everything that follows.
    # The (?:...) non-capturing group is for the optional slash or space after the credit.
    # Example: (\bFILE\s?)(.*)

    # Look for the core credit at the start or after a space/slash.
    match = re.search(r'\b(' + credit_group + r')\b(?:/|\s|$)', normalized_credit)

    if match:
        matched_term = match.group(1).strip()

        # Aggressive Cutoff: Find the position immediately after the matched term
        # and forcefully return only the text up to that point.

        # Calculate the position right after the matched credit term
        start_pos = match.start(1)
        end_pos = match.end(1)

        # We need to ensure we capture the full word, and nothing more.
        clean_text = normalized_credit[start_pos:end_pos].strip()

        # Perform final cleanup to remove trailing words like 'BY' or a slash, but only if they
        # immediately follow the term in the original raw text.

        # Remove common prefixes from the clean result if they were accidentally included
        for prefix in ['BY', 'FOR', 'FROM']:
             if clean_text.startswith(prefix + ' '):
                 clean_text = clean_text[len(prefix) + 1:].strip()

        return clean_text

    # 2. Fallback: If no known credit term was found
    for prefix in ['BY', 'FOR', 'FROM', 'PHOTO:']:
        if normalized_credit.startswith(prefix):
            normalized_credit = normalized_credit[len(prefix):].strip()

    # Return the first 3 words of the fallback
    return ' '.join(normalized_credit.split()[:3]).strip().strip('/')


# --- The Main Article Extraction Function ---
# ===== BEGIN REPLACE: page_to_articles_hybrid (v5 — deterministic assembly) =====
def page_to_articles_hybrid(page, page_index):
    """
    Layout-first + reporter-driven extraction for The Star articles.
    Refined for robust block splitting, title filtering, and deduplication.
    """

    use_ocr = not page_has_selectable_text(page)
    if use_ocr:
        logger.info(f"Page {page_index} needs OCR")
        page_text = ocr_page(page)
        header = extract_header(page_text, page_index)
        blocks_raw = [{"text": p.strip(), "bbox": None, "block": None} for p in page_text.split("\n\n") if len(p.strip())>40]
    else:
        page_text = page.get_text("text")
        header = extract_header(page_text)
        blocks_raw = split_blocks_layout(page)

    # --- 1. Refine Blocks: Split on Internal Bylines (Robust Split) ---
    blocks = []
    # Regex for a byline: Optional newline/start, whitespace, "BY" + (2 to 4 uppercase words), optional whitespace/newline
    byline_split_pattern = re.compile(r'(?:\n|^)\s*(BY\s+(?:[A-Z]+\s*){2,4})\s*(?:\n|$)', re.IGNORECASE)

    for b in blocks_raw:
        raw_txt = b.get("text", "")
        if byline_split_pattern.search(raw_txt):
            parts = byline_split_pattern.split(raw_txt)

            # First part (text before the first byline)
            if parts[0].strip():
                blocks.append({"text": parts[0].strip(), "bbox": b.get("bbox"), "block": b.get("block")})

            # Iterate over the rest in pairs (Byline, Text)
            for i in range(1, len(parts), 2):
                byline_txt = parts[i]
                body_txt = parts[i+1] if i+1 < len(parts) else ""
                full_chunk = f"{byline_txt}\n{body_txt}"
                if full_chunk.strip():
                    blocks.append({"text": full_chunk.strip(), "bbox": b.get("bbox"), "block": b.get("block")})
        else:
            blocks.append(b)

    articles = []
    page_text_norm = normalize_page_text_for_matching(page_text).lower()
    compiled_names = [(r, create_name_pattern_flexible(r)) for r in REPORTERS_LIST]

    # --- Layout-first pass ---
    for blk in blocks:
        raw = blk.get("text", "")
        if len(raw.split()) < 12:
            continue

        # --- 2. Title Fix: Prevent "BY [NAME]" from being the title ---
        title = extract_title_smart(blk)
        # CRITICAL FILTER: If the title looks like a byline, reject it.
        if title and re.match(r'^\s*BY\s+', title, re.IGNORECASE):
            title = None

        # Backward search for title (also filtering for bylines)
        if not title:
            try:
                idx_blk = blocks.index(blk)
            except ValueError:
                idx_blk = None
            if idx_blk is not None:
                # Look backwards
                for back in range(max(0, idx_blk-6), idx_blk):
                    # Check candidate block for title, strictly ignoring bylines
                    t2 = extract_title_smart(blocks[back])
                    if t2 and not re.match(r'^\s*BY\s+', t2, re.IGNORECASE):
                        title = t2
                        break
                # Final fallback for top headlines (also filtering)
                if not title:
                    for back in range(0, min(6, len(blocks))):
                        t2 = extract_title_smart(blocks[back])
                        if t2 and len(t2.split()) <= 20 and not re.match(r'^\s*BY\s+', t2, re.IGNORECASE):
                            title = t2
                            break

        # --- Detect candidate bylines ---
        lines = [l.strip() for l in raw.splitlines() if l.strip()]
        candidate_bylines = []
        for ln in lines[:14]:
            L = ln.strip()
            if re.match(r'^\s*BY\b', L, re.IGNORECASE):
                candidate_bylines.append(L)
                continue
            if re.match(r'^([A-Z][a-z]+(?:\s+[A-Z][a-z]+){1,3})$', L):
                candidate_bylines.append(L)
                continue
            if L.isupper() and 2 <= len(L.split()) <= 6:
                candidate_bylines.append(L)
                continue

        # --- Match candidate bylines ---
        found_reporters = []
        for cand in candidate_bylines:
            mr = match_reporter(cand)
            if any(s > 0 for s in mr["match_scores"]):
                found_reporters.append((cand, mr))

        # --- Fallback match in block ---
        if not found_reporters:
            for r, pat in compiled_names:
                if pat.search(raw):
                    idx = REPORTERS_LIST.index(r)
                    row = REPORTERS_DF.iloc[idx]
                    mr = {
                        "matched_names": [row["Reporter_norm"]],
                        "reporter_categories": [row.get("Reporter_Category", None)],
                        "publication_sections": [row.get("Publication_Section", None)],
                        "match_scores": [100]
                    }
                    found_reporters.append((row["Reporter_norm"], mr))

        # --- Build article(s) ---
        if found_reporters:
            # ... Reporter aggregation logic ...

            matched_names = []
            cats = []
            pubs = []
            scores = []
            byline_raw_combined = []
            for cand, mr in found_reporters:
                byline_raw_combined.append(cand)
                matched_names.extend([n for n in mr["matched_names"] if n])
                cats = [str(c).strip() for c in cats if pd.notna(c) and str(c).strip()]
                pubs = [str(p).strip() for p in pubs if pd.notna(p) and str(p).strip()]
                scores.extend([s for s in mr["match_scores"]])

            photo_credit = detect_bold_photo_credit_from_block(blk) or detect_photo_credit_anywhere(raw)

            # Determine Types
            # Determine Types
            wc = len(raw.split())

            # Specific handler for "Handout" reporter false positives
            if len(matched_names) == 1 and matched_names[0].upper() in ["HANDOUT", "FILE", "AP", "REUTERS"]:
                 art_type = "PHOTO"
            elif wc > 250:
                # If word count is high, it's definitely an article
                art_type = "MULTI-AUTHOR" if len(matched_names) > 1 else "SINGLE-AUTHOR"
            elif wc > 100:
                # If word count is moderate, check for photo credit
                if photo_credit:
                    art_type = "PHOTO" # Treat as a long caption/short photo story
                else:
                    art_type = "MULTI-AUTHOR" if len(matched_names) > 1 else "SINGLE-AUTHOR"
            elif photo_credit or wc < 100:
                # If short, or has a photo credit, classify as PHOTO unless it's the only text block
                art_type = "PHOTO"
            elif matched_names:
                art_type = "MULTI-AUTHOR" if len(matched_names) > 1 else "SINGLE-AUTHOR"
            else:
                art_type = "PHOTO" if photo_credit else "SINGLE-AUTHOR"

            # --- Strict Body Extension (Stop if new article starts) ---
            body = raw
            try:
                idx_blk = blocks.index(blk)
            except ValueError:
                idx_blk = None
            if idx_blk is not None:
                for j in range(idx_blk+1, min(idx_blk+4, len(blocks))):
                    follow = blocks[j]
                    follow_raw = follow.get("text", "")

                    # STOP CHECK 1: If next block contains a Reporter Pattern or starts with "BY "
                    if any(pat.search(follow_raw) for _, pat in compiled_names) or re.match(r'^\s*BY\s+', follow_raw, re.IGNORECASE):
                        break

                    # STOP CHECK 2: If next block has a title (not a byline)
                    t_follow = extract_title_smart(follow)
                    if t_follow and not re.match(r'^\s*BY\s+', t_follow, re.IGNORECASE):
                        break

                    if len(follow_raw.split()) < 20:
                        continue

                    body += "\n\n" + follow_raw

            art = {
                "header": header,
                "article_title": title,
                "byline_raw": "; ".join(byline_raw_combined),
                "matched_reporter": "; ".join(matched_names) if matched_names else None,
                "reporter_category": "; ".join(cats) if cats else None,
                "publication_section_from_reporter": "; ".join(pubs) if pubs else None,
                "reporter_match_scores": ";".join([str(s) for s in scores]) if scores else None,
                "article_type": art_type,
                "photo_credit": photo_credit,
                "body": body
            }
            articles.append(art)

    # --- Fallback: scan page text for REPEAT reporters (Find all instances) ---
    extracted_bodies = [a.get("body", "") for a in articles]

    for r in REPORTERS_LIST:
        pat = create_name_pattern_flexible(r)

        for m in pat.finditer(page_text_norm):
            start = m.start()

            # Context check: Is this hit already inside an extracted article? (Snippet must be > 50 chars)
            snippet_check = page_text_norm[start:start+50]
            if any(snippet_check in normalize_page_text_for_matching(b).lower() for b in extracted_bodies):
                continue

            remaining = page_text_norm[start:]

            # Heuristic stops (email, horizontal rule, next byline)
            limit = len(remaining)
            email_m = re.search(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}\b', remaining)
            horiz_m = re.search(r'-{2,}', remaining)
            next_byline = re.search(r'\n\s*BY\s+', remaining[20:], re.IGNORECASE)

            if email_m: limit = min(limit, email_m.start())
            if horiz_m: limit = min(limit, horiz_m.start())
            if next_byline: limit = min(limit, next_byline.start() + 20)

            body = remaining[:limit].strip()

            if len(body.split()) < 20:
                continue

            # ... Fallback article creation logic ...

            try:
                idx = REPORTERS_LIST.index(r)
                row = REPORTERS_DF.iloc[idx]
                matched_name = row["Reporter_norm"]
                cat = row.get("Reporter_Category", None)
                pubsec = row.get("Publication_Section", None)
            except Exception:
                matched_name, cat, pubsec = r, None, None

            photo_credit_fallback = detect_photo_credit_anywhere(body)
            wc_fallback = len(body.split())

            if wc_fallback > 200:
                art_type = "SINGLE-AUTHOR"
            elif photo_credit_fallback and wc_fallback < 80:
                art_type = "PHOTO"
            else:
                art_type = "SINGLE-AUTHOR"

            art = {
                "header": header,
                "article_title": None,
                "byline_raw": matched_name,
                "matched_reporter": matched_name,
                "reporter_category": cat,
                "publication_section_from_reporter": pubsec,
                "reporter_match_scores": 100,
                "article_type": art_type,
                "photo_credit": photo_credit_fallback,
                "body": body
            }
            articles.append(art)
            extracted_bodies.append(body)

    # --- Dedupe articles (FINAL FIX: Robust 200-char snippet) ---
    unique = []
    seen = set()
    for a in articles:
        # Use a long snippet (200 chars) to distinguish between overlapping or closely spaced duplicate blocks
        body_snippet = (a.get("body","")[:200] or "").strip().lower()
        key = ((a.get("matched_reporter") or "").lower(), body_snippet)

        if key in seen:
            continue
        seen.add(key)
        unique.append(a)

    # --- Enrich articles and Clean Photo Credit ---
    rows = []
    for a in unique:
        hdr = a.get("header", {})
        title = a.get("article_title") or (a.get("body"," ").splitlines()[0] if a.get("body") else None)
        topic = match_topic(a.get("body"," "), section_hint=hdr.get("news_section"))
        word_count = len(a.get("body"," ").split())

        raw_photo_credit = a.get("photo_credit")
        cleaned_photo_credit = clean_photo_credit(raw_photo_credit)

        row = {
            "date": hdr.get("date"),
            "page_number": page_index,
            "news_section": hdr.get("news_section"),
            "article_title": title,
            "byline_raw": a.get("byline_raw"),
            "reporter_matched": a.get("matched_reporter"),
            "reporter_match_scores": a.get("reporter_match_scores"),
            "article_type": a.get("article_type"),
            "photo_credit": cleaned_photo_credit,
            "topic": topic,
            "word_count": word_count,
            "excerpt": a.get("body"," ")[:400]
        }
        rows.append(row)

    return rows
# ===== END REPLACE: page_to_articles_hybrid =====

# ---------------- Main extraction ----------------
def extract_newspaper_metadata(pdf_path):
    logger.info(f"Opening PDF: {pdf_path}")
    doc = fitz.open(pdf_path)
    all_rows = []
    for i in range(len(doc)):
        page = doc.load_page(i)
        logger.info(f"Processing page {i+1}/{len(doc)}")
        # decide OCR or not & get header
        raw_text = page.get_text("text") if page_has_selectable_text(page) else ocr_page(page)
        header = extract_header(raw_text, i+1)
        rows = page_to_articles_hybrid(page, i+1)
        # set header fallback if missing per row
        for r in rows:
            if not r.get("date"): r["date"] = header.get("date")
            if not r.get("page_number"):
              r["page_number"] = header.get("page_number") #or page_index  # page_index is the (i+1) you pass to page_to_articles_hybrid
            if not r.get("publication"): r["publication"] = header.get("publication")
            if not r.get("news_section"): r["news_section"] = header.get("news_section")
        all_rows.extend(rows)
    df = pd.DataFrame(all_rows)
    return df

# --------------- Run & Export ---------------
if __name__ == "__main__":
    os.makedirs(os.path.dirname(OUTPUT_CSV_DRIVE_PATH), exist_ok=True)
    df = extract_newspaper_metadata(PDF_PATH)
    if df.empty:
        logger.warning("No articles extracted.")
    else:
        try:
            df.to_csv(OUTPUT_CSV_DRIVE_PATH, index=False)
            logger.info(f"Wrote CSV to {OUTPUT_CSV_DRIVE_PATH}")
        except Exception as e:
            logger.error(f"Error writing CSV: {e}")
            df.to_csv("newspaper_metadata_v4_fallback.csv", index=False)
            logger.info("Wrote fallback CSV locally.")
    print(df.head(20))

                         date  page_number        news_section  \
0                        None            1            BIG READ   
1                        None            1            BIG READ   
2   Monday, September 1, 2025            2                None   
3   Monday, September 1, 2025            2                None   
4   Monday, September 1, 2025            2                None   
5   Monday, September 1, 2025            2                None   
6   Monday, September 1, 2025            3  CORRIDORS OF POWER   
7   Monday, September 1, 2025            4                NEWS   
8   Monday, September 1, 2025            4                NEWS   
9                        None            5                None   
10                       None            5                None   
11                       None            6                NEWS   
12                       None            6                NEWS   
13                       None            7                NEWS   
14        